In [1]:
import pandas as pd
import numpy as np
import os

In [3]:
input_file = "../dirty_data/S&P_500_Historische_Daten.csv"
output_file = "../clean_data/SP500_Cleaned.csv"


df = pd.read_csv(input_file)
df_clean = df.copy()

In [4]:
# Spalten umbenennen
df_clean = df_clean.rename(columns={
    'Datum': 'Date', 
    'Zuletzt': 'Close', 
    'Eröffn.': 'Open',
    'Hoch': 'High', 
    'Tief': 'Low', 
    'Vol.': 'Volume', 
    '+/- %': 'Change_Percent'
})

In [5]:
# Zahlenformate korrigieren: 1.000,00 -> 1000.00
cols_to_fix = ['Close', 'Open', 'High', 'Low', 'Change_Percent']

for col in cols_to_fix:
    # Zeichen ersetzen
    df_clean[col] = (df_clean[col].astype(str)
                     .str.replace('.', '', regex=False)
                     .str.replace(',', '.', regex=False)
                     .str.replace('%', '', regex=False))
    
    # In float umwandeln
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')


In [6]:

# Datum konvertieren und als Index setzen
df_clean['Date'] = pd.to_datetime(df_clean['Date'], format='%d.%m.%Y')
df_clean = df_clean.set_index('Date').sort_index()


In [7]:

# Volumen entfernen 
if 'Volume' in df_clean.columns:
    df_clean = df_clean.drop(columns=['Volume'])

# Duplikate im Datum entfernen
df_clean = df_clean[~df_clean.index.duplicated(keep='first')]


# Leere Zeilen entfernen (falls durch Konvertierung Fehler entstanden sind)
df_final = df_clean.dropna()


In [8]:
print("\n--- Fertiger Datensatz ---")
print(df_final.head())


df_final.to_csv(output_file)


--- Fertiger Datensatz ---
              Close     Open     High      Low  Change_Percent
Date                                                          
2022-11-28  3963.94  4005.36  4012.27  3955.77           -1.54
2022-11-29  3957.63  3964.19  3976.77  3937.65           -0.16
2022-11-30  4080.11  3957.18  4080.11  3938.58            3.09
2022-12-01  4076.57  4087.14  4100.51  4050.87           -0.09
2022-12-02  4071.70  4040.17  4080.48  4026.63           -0.12
